In [1]:
# Utility
import re
import sys
import copy
import arrow
from pprint      import pprint
from collections import abc
from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache, reduce
from itertools   import permutations, combinations, chain, cycle, product, islice, zip_longest
from heapq       import heappop, heappush
from time        import sleep
from multiprocess import Process, Queue, Pipe
from math import atan2, degrees, ceil, floor
from importlib import import_module
from IPython.display import clear_output
import random

def contents(year, day, part):
    with open(f'./y{year}/day{day}/part{part}') as f:
        return f.read().splitlines()

def take(n, iterable, default=None):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n, default))

def nth(iterable, n, default=None):
    "Returns the nth item or a default value"
    return next(islice(iterable, n, None), default)

first = lambda iterable: nth(iterable, 0)
map_ints = lambda ints: list(map(int, list(ints)))
get_ints = lambda line: map_ints(re.findall(r'-?\d+', line))
cat = ''.join

def grouper(iterable, n, fillvalue=None):
    """Collect data into fixed-length chunks:
    grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"""
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

def overlapping(iterable, n):
    """Generate all (overlapping) n-element subsequences of iterable.
    overlapping('ABCDEFG', 3) --> ABC BCD CDE DEF EFG"""
    if isinstance(iterable, abc.Sequence):
        yield from (iterable[i:i+n] for i in range(len(iterable) + 1 - n))
    else:
        result = deque(maxlen=n)
        for x in iterable:
            result.append(x)
            if len(result) == n:
                yield tuple(result)
                
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    return overlapping(iterable, 2)

def sequence(iterable, type=tuple):
    "Coerce iterable to sequence: leave alone if already a sequence, else make it `type`."
    return iterable if isinstance(iterable, abc.Sequence) else type(iterable)

def join(iterable, sep=''):
    "Join the items in iterable, converting each to a string first."
    return sep.join(map(str, iterable))
                
def powerset(iterable):
    "Yield all subsets of items."
    items = list(iterable)
    for r in range(len(items)+1):
        for c in combinations(items, r):
            yield c

################ 2-D points implemented using (x, y) tuples

origin = (0, 0)

def X(point): return point[0]
def Y(point): return point[1]

xy_HEADINGS = xy_UP, xy_LEFT, xy_DOWN, xy_RIGHT = (0, -1), (-1, 0), (0, 1), (1, 0)

def xy_turn_right(heading): return xy_HEADINGS[xy_HEADINGS.index(heading) - 1]
def xy_turn_around(heading):return xy_HEADINGS[xy_HEADINGS.index(heading) - 2]
def xy_turn_left(heading):  return xy_HEADINGS[xy_HEADINGS.index(heading) - 3]


def R(point): return point[0]
def C(point): return point[1]

rc_HEADINGS = rc_UP, rc_LEFT, rc_DOWN, rc_RIGHT = (-1, 0), (0, -1), (1, 0), (0, 1)

def rc_turn_right(heading): return rc_HEADINGS[rc_HEADINGS.index(heading) - 1]
def rc_turn_around(heading):return rc_HEADINGS[rc_HEADINGS.index(heading) - 2]
def rc_turn_left(heading):  return rc_HEADINGS[rc_HEADINGS.index(heading) - 3]

def neighbors4(point): 
    "The four neighboring squares."
    x, y = point
    return (          (x, y-1),
            (x-1, y),           (x+1, y), 
                      (x, y+1))

def neighbors8(point): 
    "The eight neighboring squares."
    x, y = point 
    return ((x-1, y-1), (x, y-1), (x+1, y-1),
            (x-1, y),             (x+1, y),
            (x-1, y+1), (x, y+1), (x+1, y+1))

def add(A, B): 
    "Element-wise addition of two n-dimensional vectors."
    return mapt(sum, zip(A, B))

def cityblock_distance(P, Q=origin): 
    "Manhatten distance between two points."
    return sum(abs(p - q) for p, q in zip(P, Q))

def distance(P, Q=origin): 
    "Straight-line (hypotenuse) distance between two points."
    return sum((p - q) ** 2 for p, q in zip(P, Q)) ** 0.5

def king_distance(P, Q=origin):
    "Number of chess King moves between two points."
    return max(abs(p - q) for p, q in zip(P, Q))

################ functional

def mapt(fn, *args): 
    "Do a map, and make the results into a tuple."
    return tuple(map(fn, *args))

################ A* and Breadth-First Search (tracking states, not actions)

def always(value): return (lambda *args: value)

def Astar(start, moves_func, h_func, cost_func=always(1)):
    "Find a shortest sequence of states from start to a goal state (where h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    Path      = lambda s: ([] if (s is None) else Path(previous[s]) + [s])
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(s)
        for s2 in moves_func(s):
            g = path_cost[s] + cost_func(s, s2)
            if s2 not in path_cost or g < path_cost[s2]:
                heappush(frontier, (g + h_func(s2), s2))
                path_cost[s2] = g
                previous[s2] = s
                
def AstarSteps(*args, **kwargs):
    return len(Astar(*args, **kwargs)) - 1

def bfs(start, moves_func, goals):
    "Breadth-first search"
    goal_func = (goals if callable(goals) else lambda s: s in goals)
    return Astar(start, moves_func, lambda s: (0 if goal_func(s) else 1))

################ timing info

from time import perf_counter

def timeit(method):
    def timed(*args, **kw):
        ts = perf_counter()
        result = method(*args, **kw)
        te = perf_counter()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print('%r  %2.2f ms' % (method.__name__, (te - ts) * 1000))
        return result
    return timed


In [3]:
from y2019.machine import Machine

class Droid:
    def __init__(self, program):
        self.program = program
        self.m = Machine(self.program, self.move, self.status, always(False))
        self.grid = {}
        self.start = (0, 0)
        self.dir = None
        self.grid[self.start] = 'X'
        self.current = self.start
        self.destination = None
        self.shortest = None
        self.count = 0
        self.translate = {
            xy_UP: 1,
            xy_DOWN: 2,
            xy_LEFT: 3,
            xy_RIGHT: 4,
        }
        self.options = list(self.translate.keys())
        
    def run(self):
        try:
            self.m.run()
        except:
            raise
            
    def move(self):
        self.count += 1
        d = random.choice(self.options)
        while(self.grid.get(add(self.current, d)) == '#'):
            d = random.choice(self.options)
            
        if self.grid.get(add(self.current, d) == '.'):
            for e in random.choices(self.options, k=4):
                if self.grid.get(add(self.current, e) is None):
                    d = e
                    break
        
        if (self.count // 100 == self.count / 100):
            print('.', end='')
        self.dir = d
        return self.translate[d]
            
    def grid_size(self):
        lx, hx, ly, hy = None, None, None, None
        for x, y in self.grid.keys():
            lx = x if lx is None or x < lx else lx
            ly = y if ly is None or y < ly else ly
            hx = x if hx is None or x > hx else hx
            hy = y if hy is None or y > hy else hy
        return lx, ly, hx, hy
            
    def display(self):
        lx, ly, hx, hy = self.grid_size()
        clear_output()
        for y in range(ly, hy + 1):
            for x in range(lx, hx + 1):
                pixel = self.grid.get((x, y), ' ')
                if (x, y) == self.current:
                    pixel = "@"
                print(pixel, end='')
            print('')
        print(len(self.grid.keys()), 'tiles')
        print('')
        sleep(0.3)
        
    def get_path(self):
        def moves(s):
            return [d for d in neighbors4(s) if self.grid.get(d) not in [None, '#']]
        def goal(s):
            return 0 if s == self.start else 1
        return len(Astar(self.destination, moves, goal))
    
    def get_time(self):
        count = 0
        while True:
            count += 1
            change = []
            for xy, value in self.grid.items():
                if value == 'O':
                    for n in neighbors4(xy):
                        if self.grid.get(n) == '.':
                            change.append(n)
            for xy in change:
                self.grid[xy] = 'O'
            done = True
            for value in self.grid.values():
                if value == '.':
                    done = False
                    break
            if done:
                print('minutes', count)
                return
                    
    
    def status(self, value):
        # 0 - wall ahead, hasn't moved
        # 1 - moved as requested
        # 2 - moved, found Oxygen
        if value == 0: 
            self.grid[add(self.current, self.dir)] = "#"
        if value in [1,2]:
            self.current = add(self.current, self.dir)
            if self.grid.get(self.current) != 'X':
                self.grid[self.current] = '.'
        if value == 2:
            self.grid[self.current] = 'O'
            self.destination = self.current
        if self.destination is not None:
            d = self.get_path()
            if self.shortest is None or d < self.shortest:
                self.shortest = d
                print(d-1)
                if self.shortest == 259:
                    self.get_time()
                    1/0
        
raw = contents(2019, 15, 1)
data = map_ints(raw[0].split(','))

d = Droid(data)
d.run()
        

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

ZeroDivisionError: division by zero

In [ ]:
pic = """                       ######### 
                              #.........#
                             ##.#######.#
                            #...#.....#.#
                            #.###.###.#.#
                            #.#...#...#.#
 #####               # # ####.###.#.###.#
#.....#             ....#.........#.#.#.#
#.###.#          ######.#.#########.#.#.#
#...#.#         #.....#...#.....#.....#.#
 ##.#.######## ##.###.#####.###.#######.#
#.#.#.#.......#...#.......#.#.#.......#.#
#.#.#.#.#####.#.#########.#.#.#.#####.#.#
#.#.#.#..@#.....#....X#.#.#.#...#...#...#
#.#.#.###.###.###.#####.#.#.#####.#.#### 
#.#.#...#...#.#...#...#...#...#...#.....#
#.#.###.###.###.###.###.###.#.#.## ####.#
#.#...#.....#...#.......#...#...#.#.....#
#.###.#######.###.###########.###.#.###.#
#...#.....#...#.#.#.........#.#...#...#.#
 ##.#####.#.###.#.#.#.#####.#.#.#####.#.#
#.......#...#...#.#.#...#...#...#.....#.#
#.#####.#####.#.#.#####.#.#####.#.#####.#
#.#.....#.....#.#.......#.#...#.#.#...#.#
#.#####.#.#####.#########.#.#.#.#.###.#.#
#.#...#O#.#...#.....#...#...#.#.#...#...#
#.#.#.###.#.#####.#.#.#.#####.#####.###.#
#...#...#.#.#.....#...#.....#.....#...#.#
#.## ##.#.#.#.#### ##.#####.#####.#.#.## 
#...#.#.#...#.#...#...#...#.#.....#.#...#
 ##.#.#.###.#.#.#.#####.#.###.#########.#
#.....#.....#...#.......#...............#
 ##### ##### ### ####### ############### """
for y, row in enumerate(pic.split('\n'):
    for x

In [3]:
"""
Day 14
"""
@timeit
def day14():
    def split(item):
        i, n = item.split(' ')
        return int(i), n
    
    def ore_count(data, objective=1):
        reactions = {}
        for reaction in data:
            input, output = reaction.split(' => ')
            inputs = []
            for item in input.split(', '):
                inputs.append(split(item))
            yields, o_name = split(output)
            reactions[o_name] = (yields, inputs)
            
        q = [(False, objective, 'FUEL')]
        waste = defaultdict(lambda:0)
        ore = 0
        while len(q) > 0:
            flag, needed, chem = q.pop()
            if chem == 'ORE':
                ore += needed
                continue
            if flag:
                waste[chem] += needed
                continue
                
            reuse = min(needed, waste[chem])
            needed -= reuse
            waste[chem] -= reuse
            
            yields, inputs = reactions[chem]
            batches = ceil(needed/yields)
            
            # waste needs to be processed after the reaction is complete
            #  So first on the stack
            q.append((True, (batches * yields) - needed, chem))
            
            for amount, name in inputs:
                q.append((False, batches * amount, name))
        return ore
    
    raw = contents(2019,14,1)
    
    @timeit
    def part1():
        print(ore_count(raw[:]))

    @timeit
    def part2():
        target = 1000000000000
        high = 1
        low = 1
        
        while ore_count(raw[:], high) < target:
            low = high
            high *=2
        
        while low + 1 < high:
            mid = (low + high) // 2
            ore = ore_count(raw[:], mid)
            if ore > target:
                high = mid
            if ore < target:
                low = mid
        print(low)
        

    part1()
    part2()
day14()

1582325
'part1'  1.99 ms
2267486
'part2'  69.08 ms
'day14'  77.49 ms


In [3]:
"""
Day 13

380
'part1'  54.52 ms
18647
'part2'  2284.40 ms
'day13'  2347.69 ms
"""
@timeit
def day13():
    from y2019.machine import Machine
            
    class Arcade:
        def __init__(self, program):
            self.program = program
            self.machine = Machine(self.program, self.input, self.output, self.halt)
            self.queue = []
            self.grid = defaultdict(lambda: '')
            self.ball = deque([], 2)
            self.paddle = None
            self.score = None
            self.TILES = frozenset(range(5))
            self.BALL = 4
            self.PADDLE = 3

        def run(self):
            self.machine.run()

        def input(self):
            if len(self.ball) < 2:
                return 1 if self.ball[0] > self.paddle else -1
            ox, cx, px = self.ball[0], self.ball[1], self.paddle
            dx = cx - ox # > 0 going right
            if dx > 0 and (px < cx or abs(dx) > 1):
                return 1
            if dx < 0 and (px > cx or abs(dx) > 1):
                return -1
            return 0

        def halt(self):
            if self.score:
                print(self.score)

        def output(self, value):
            self.queue.append(value)
            if len(self.queue) < 3:
                return

            x, y, v = self.queue
            self.queue = []

            if v not in self.TILES:
                self.score = v
                return

            self.grid[(x,y)] = v

            if v == self.BALL:
                self.ball.append(x)
            if v == self.PADDLE:
                self.paddle = x
    
    raw = contents(2019,13,1)
    data = map_ints(raw[0].split(','))

    @timeit
    def part1():
        a = Arcade(data)
        a.run()
        print(Counter(a.grid.values())[2])
    
    @timeit
    def part2():
        data[0] = 2
        a = Arcade(data)
        a.run()
    
    part1()
    part2()
    
day13()


380
'part1'  75.71 ms
18647
'part2'  2281.93 ms
'day13'  2364.79 ms


In [51]:
"""
Day 12
"""
@timeit
def day12():

    class Moon:
        def __init__(self, pos):
            self.pos = pos
            self.vel = [0,0,0]
            self.n = ['x', 'y', 'z']
            self.orig = pos[:]

        def __str__(self):
            p = "pos=<" + ", ".join([f"{n}={self.pos[i]:3}" for i, n in enumerate(self.n)]) + ">"
            v = "vel=<" + ", ".join([f"{n}={self.vel[i]:3}" for i, n in enumerate(self.n)]) + ">"
            return f"{p}, {v}"

        def move(self):
            for i in range(3):
                self.pos[i] += self.vel[i]

        def energy(self):
            return sum(map(abs, self.pos)) * sum(map(abs, self.vel))

    def gravity(m1, m2):
        for i in range(3):
            if m1.pos[i] < m2.pos[i]:
                m1.vel[i] += 1
                m2.vel[i] -= 1
            elif m1.pos[i] > m2.pos[i]:
                m1.vel[i] -= 1
                m2.vel[i] += 1

    raw = contents(2019, 12, 1)
    data = [get_ints(l) for l in raw]
    moons = [Moon(l) for l in data]

    def part1():
        for m in moons:
            print(m)
        print("--")

        for _ in range(1000):
            for m1, m2 in combinations(moons, 2):
                gravity(m1, m2)
            for m in moons:
                m.move()

        s = 0
        for m in moons:
            print(m.energy())
            s += m.energy()
        print(s)

    def part2():
        def gcd(a, b):
            if a < b:
                a, b = b, a
            while b:
                a, b = b, a % b
            return a

        def compute_lcm(x, y):
            return (x*y)//gcd(x,y)

        vels = [(0, 0, 0)] * len(moons)
        intervals = {}
        for axis in range(3):
            seen = set()
            step = 0
            vals = [m.pos[axis] for m in moon]
            speeds = [0] * len(vals)

            while True:
                key = tuple(vals) + tuple(speeds)
                if key in seen:
                    intervals[axis] = step
                    break
                else:
                    seen.add(key)
                new_s
    part1()
    part2()
day12()
    

In [24]:
inp = """<x=-19, y=-4, z=2>
<x=-9, y=8, z=-16>
<x=-4, y=5, z=-11>
<x=1, y=9, z=-13>"""

inp = inp.split("\n")

coords = [
    line[1:-1].split(", ")
    for line in inp
]

posns = [
    tuple([int(coord[2:]) for coord in line])
    for line in coords
]

vels = [
    (0, 0, 0)
] * len(posns)

# Find the interval along each axis as each one is indepenent
intervals = {}
for axis in range(3):
    seen = set()
    step = 0

    vals = [pos[axis] for pos in posns]
    speeds = [0] * len(vals)

    while True:
        key = tuple(vals) + tuple(speeds)
        if key in seen:
            intervals[axis] = step
            break
        else:
            seen.add(key)

        new_speeds = []
        for i, (pos_a, vel_a) in enumerate(zip(vals, speeds)):
            for j, pos_b, in enumerate(vals):
                if pos_a < pos_b:
                    vel_a += 1
                elif pos_a > pos_b:
                    vel_a -= 1
            new_speeds.append(vel_a)

        speeds = new_speeds

        vals = [
            val + speed
            for val, speed in zip(vals, speeds)
        ]

        step += 1

def gcd(a, b):
    if a < b:
        a, b = b, a
    while b:
        a, b = b, a % b
    return a

print(intervals)

def compute_lcm(x, y):
    lcm = (x*y)//gcd(x,y)
    return lcm

print(compute_lcm(intervals[0], compute_lcm(intervals[1], intervals[2])))

{0: 161428, 1: 113028, 2: 231614}
528250271633772


In [4]:
"""
Day 11
"""

@timeit
def day11():
    from y2019.machine import Machine

    class Robot:
        def __init__(self, program):
            self.direction = xy_UP
            self.current = origin
            self.grid = {}
            self.paint_cmd = True
            self.machine = Machine(program, self.camera, self.output, self.halt)
            self.counter = 0

        def run(self):
            self.machine.run()

        def camera(self):
            return self.grid.get(self.current, 0)

        def output(self, value):
            if self.paint_cmd:
                self.paint(value)
                self.paint_cmd = False
            else:
                self.move(value)
                self.paint_cmd = True

        def paint(self, color):
            if self.current not in self.grid.keys():
                self.counter += 1
            self.grid[self.current] = color

        def move(self, direction):
            func = xy_turn_left if direction == 0 else xy_turn_right
            self.direction = func(self.direction)
            self.current = add(self.current, self.direction)

        def halt(self):
            print(self.counter)

        def get(self):
            return self.grid
    
    raw = map_ints(contents(2019,11,1)[0].split(','))
    
    @timeit
    def part1():
        program = raw[:]        
        r = Robot(program)
        r.run()
        
        result = r.get()
        bx, by = 0, 0
        lx, ly = 0, 0
        for k in result.keys():
            x, y = k
            bx = max([bx, x])
            by = max([by, y])
            lx = min([lx, x])
            ly = min([ly, y])
            
        for y in range(ly-2, by+2):
            for x in range(lx-2, bx+2):
                item = result.get((x,y), 0)
                print(' ' if item == 0 else '\u2588', end='')
            print('')
    
    @timeit
    def part2():
        program = raw[:]        
        r = Robot(program)
        r.grid[r.current] = 1
        r.run()
        
        result = r.get()
        mx, my = 0, 0
        for k in result.keys():
            if X(k) > mx:
                mx = X(k)
            if Y(k) > my:
                my = Y(k)
        print(mx, my)
        for y in range(my+1):
            for x in range(mx+1):
                item = result.get((x,y), 0)
                print(' ' if item == 0 else '\u2588', end='')
            print('')
    
    part1()
    part2()
    
day11()

1564
                                                             
                                                             
                                    ████                     
                                    █   █  ██                
                                    ██████  ██               
                                    ██   █   █               
                                   █ █ █████ █████  ██       
                           ██   ██ █ █ █ ██  █ █ █████       
                          █    ███ ██ ███ █   ██ █           
                          ██████  █  █████   █   █ █ █       
                             ██   █  █████ █ ██  █  █ █      
                                 █     █    ███████ ███      
                                ██ █  ██   █████  █ █        
                                █ █   ██  ██ █  █  █ █       
                                    █  █     █ ██ █          
                                 █████     █ ████ ██         
   

In [56]:
"""
Day 10
"""

@timeit
def day10():
    def grid_list_to_dict(data, func=always(False)):
        rv1 = {}
        rv2 = set()
        maxx = None
        for iy, y in enumerate(data):
            for ix, x in enumerate(y):
                rv1[(ix, iy)] = x
                if func(x):
                    rv2.add((ix, iy))
            maxx = ix
        return rv1, rv2, maxx+1, iy+1

    def display_grid(data, mx, my, func=always(False)):
        for y in range(my):
            for x in range(mx):
                item = data[(x, y)]
                if func((x, y)):
                    item = '@'
                print(item, end='')
            print('')

    def collinear(a, b, c):
        ax, ay = a
        bx, by = b
        cx, cy = c
        return (bx - ax) * (cy - ay) == (cx - ax) * (by - ay)

    def within(p, q, r):
        return (p <= q <= r or r <= q <= p) and (p != r or p == q == r)

    def is_on(a, b, c):
        ax, ay = a
        bx, by = b
        cx, cy = c
        return (collinear(a, b, c) 
                and (within(ax, cx, bx) 
                            if ax != bx 
                            else within(ay, cy, by)))
    
    def clock_order(center, point):
        cx, cy = center
        px, py = point
        dist = ((px - cx)**2 + (py - cy)**2)**0.5
        dx, dy = px - cx, py - cy
        rads = atan2(dx, -dy)
        degs =  degrees(rads)
        if degs < 0:
            degs = 360 + degs
        return degs, dist

    def check(a, b):
        for c in [i for i in asteriods if i not in (a, b)]:
            if is_on(a, b, c):
                return 0
        return 1

    def check_all(point):
        return sum([check(point, a) for a in asteriods if a != point])

    def findNthAsteroid(o, nth, rocks):
        removed = set([o])
        while True:
            alive = [a for a in rocks if a not in removed]
            if len(alive) == 0:
                return None
            for b in alive:
                occluded = False
                current = set([o, b])
                for c in alive:
                    if c in current: continue
                    if is_on(o, b, c):
                        occluded = True
                        break
                if occluded is False:
                    removed.add(b)
                    if len(removed)-1 == nth:
                        return b

    raw = contents(2019, 10, 1)
    grid, asteriods, mx, my = grid_list_to_dict(raw, lambda a: a != '.')
    highest = None
    
    @timeit
    def part1():
        nonlocal highest
        for asteriod in asteriods:
            tally = check_all(asteriod)
            if highest is None or tally > highest[1]:
                highest = (asteriod, tally)
        print(highest)
        
    @timeit
    def part2():
        nonlocal highest
        found = findNthAsteroid(
            highest[0], 
            200, 
            sorted(asteriods, key=lambda p: clock_order(highest[0], p))
        )
        print(found)
        print((X(found) * 100) + Y(found))
    
    part1()
    part2()
day10()

((20, 20), 292)
'part1'  26107.88 ms
(3, 17)
317
'part2'  40.67 ms
'day10'  26149.87 ms


In [5]:
"""
Day 9

3280416268
halt:  1102
'part1'  1.37 ms
80210
halt:  1102
'part2'  1155.56 ms
'day9'  1157.87 ms

Machine moved to separate file as we've been told that it's complete
"""
@timeit
def day9():
    from y2019.machine import Machine
            
    program = map_ints(contents(2019, 9, 1)[0].split(','))
    halt = always(False)
        
    @timeit
    def part1():
        m = Machine(program, always(1), print, halt)
        m.run()
    
    @timeit
    def part2():
        m = Machine(program, always(2), print, halt)
        m.run()
        
    part1()
    part2()
day9()

3280416268
'part1'  1.68 ms
80210
'part2'  1267.69 ms
'day9'  1270.39 ms


In [59]:
"""
Day 8
"""
@timeit
def day8():
    layers = list(grouper(contents(2019, 8, 1)[0], 25*6))[::-1]
    leastZero = min([Counter(l) for l in layers], key=lambda c: c['0'])
    final = defaultdict(lambda: '2')

    for layer in layers:
        for idx, pixel in enumerate(layer):
            final[idx] = pixel if pixel != '2' else final[idx]

    print(leastZero['1'] * leastZero['2'])

    for row in range(6):
        for col in range(25):
            print(' ' if final[row*25+col] == '0' else '\u2588', end='')
        print('')
day8()

1441
███  █  █ ████ ███  ███  
█  █ █  █    █ █  █ █  █ 
█  █ █  █   █  ███  █  █ 
███  █  █  █   █  █ ███  
█ █  █  █ █    █  █ █    
█  █  ██  ████ ███  █    
'day8'  28.60 ms


In [9]:
"""
Day 7
"""

@timeit
def day7():
    from y2019.machine import Machine
    
    class Amp(Process):
        def __init__(self, *args, **kwargs):
            self.program = kwargs.pop('program')
            self.inp = kwargs.pop('inp')
            self.outp = kwargs.pop('outp')
            self.phase = kwargs.pop('phase')
            self.machine = Machine(self.program, 
                                   self.input, 
                                   self.output, 
                                   always(False))
            self.inp.put(self.phase)
            
            super().__init__(*args, **kwargs)
            
        def run(self):
            self.machine.run()
            
        def input(self):
            return self.inp.get()
        
        def output(self, value):
            self.outp.put(value)
            
    raw = contents(2019, 7, 1)[0].split(',')
    ops = map_ints(raw)
        
    @timeit
    def part1():
        start = Queue()
        ab = Queue()
        bc = Queue()
        cd = Queue()
        de = Queue()
        output = Queue()
        queues = [start, ab, bc, cd, de, output]
        high = None
        highp = None
        for p in permutations(range(5)):
            amps = [
                Amp(program=ops, inp=start, outp=ab,     phase=p[0]), # A 
                Amp(program=ops, inp=ab,    outp=bc,     phase=p[1]), # B
                Amp(program=ops, inp=bc,    outp=cd,     phase=p[2]), # C
                Amp(program=ops, inp=cd,    outp=de,     phase=p[3]), # D
                Amp(program=ops, inp=de,    outp=output, phase=p[4]), # E
            ]
            [a.start() for a in amps]
            start.put(0)
            [a.join() for a in amps]
            rv = output.get()
            [a.close() for a in amps]
            if high is None or rv > high:
                high = rv
                highp = p
        [q.close() for q in queues]
        print(high, highp)
    
    @timeit
    def part2():
        high = None
        highp = None
        ea, ab, bc, cd, de = Queue(), Queue(), Queue(), Queue(), Queue()
        queues = [ea, ab, bc, cd, de]
        for p in permutations(range(5, 10)):    
            amps = [
                Amp(program=ops, inp=ea, outp=ab, phase=p[0]), # A
                Amp(program=ops, inp=ab, outp=bc, phase=p[1]), # B
                Amp(program=ops, inp=bc, outp=cd, phase=p[2]), # C
                Amp(program=ops, inp=cd, outp=de, phase=p[3]), # D
                Amp(program=ops, inp=de, outp=ea, phase=p[4]), # E
            ]
            [a.start() for a in amps]
            ea.put(0)
            [a.join() for a in amps]
            rv = ea.get()
            [a.close() for a in amps]
            if high is None or rv > high:
                high = rv
                highp = p
        [q.close() for q in queues]
        print(high, highp)
        
    
    part1()
    part2()
day7()

92663 (3, 1, 4, 2, 0)
'part1'  3838.82 ms
14365052 (7, 8, 6, 9, 5)
'part2'  4567.29 ms
'day7'  8406.84 ms


In [60]:
"""
Day 6

322508
'part1'  52.00 ms
496
'part2'  0.00 ms
'day6'  54.98 ms
"""
@timeit
def day6():
    raw = contents(2019, 6, 1)
    planets = {}

    for orbit in raw:
        planet, moon = orbit.split(')')
        planets[moon] = planet
            
    @timeit
    def part1():        
        orbits = []
        for current in planets.values():
            while current is not None:
                orbits.append(current)
                current = planets.get(current)
        print(len(orbits))
    
    @timeit
    def part2():
        o = planets['YOU']
        o_path = set()
        current = o
        while current is not None:
            o_path.add(current)
            current = planets.get(current)
        
        d = planets['SAN']
        d_path = set()
        current = d
        while current is not None and current not in o_path:
            d_path.add(current)
            current = planets.get(current)
            
        remove = planets.get(current)
        while remove is not None:
            o_path.remove(remove)
            remove = planets.get(remove)
        print(len(o_path ^ d_path)-1) # -1 since we want transitions, not states
    
    part1()
    part2()
day6()

322508
'part1'  52.36 ms
496
'part2'  0.26 ms
'day6'  62.47 ms


In [61]:
"""
Day 5

0
0
0
0
0
0
0
0
0
7265618
exit:  3
'part1'  0.00 ms
7731427
exit:  314
'part2'  1.02 ms
'day5'  1.02 ms
"""
class Machine:
    def __init__(self, ops, data):
        Instruction = namedtuple('Instruction', ['func', 'ip'])
        self.ops = ops[:]
        self.ip = 0
        self.params = {}
        self.data = data
        self.instructions = {  1: Instruction(self.add, 4), 2: Instruction(self.mul, 4),
                               3: Instruction(self.get, 2), 4: Instruction(self.put, 2),
                               5: Instruction(self.jnz, 3), 6: Instruction(self.jez, 3),
                               7: Instruction(self.clt, 4), 8: Instruction(self.ceq, 4),
                              99: Instruction(self.hlt, 0) }
        
    def run(self):
        while True:
            old_ip = self.ip
            instruction = self.get_instruction()
            if (instruction.func()):
                return
            self.ip += instruction.ip if self.ip == old_ip else 0
            
    def get_instruction(self):
        op = f"{self.ops[self.ip]}"
        self.params =  {idx: int(p) for idx, p in enumerate(op[:-2][::-1], 1)}
        return self.instructions.get(int(op[-2:]), None)
        
    def read(self, idx):
        rv = self.ops[self.ip + idx]
        return self.ops[rv] if self.params.get(idx, 0) == 0 else rv
    
    def write(self, idx, v):
        self.ops[self.ops[self.ip + idx]] = v
    
    def hlt(self):
        print("halt: ", self.ops[0])
        return True
    
    def add(self): self.write(3, self.read(1) + self.read(2))
    def mul(self): self.write(3, self.read(1) * self.read(2))
    def get(self): self.write(1, int(self.data.pop()))
    def put(self): print(self.read(1))
    def jnz(self): self.ip = self.read(2) if self.read(1) != 0 else self.ip
    def jez(self): self.ip = self.read(2) if self.read(1) == 0 else self.ip
    def clt(self): self.write(3, 1 if self.read(1) < self.read(2) else 0)
    def ceq(self): self.write(3, 1 if self.read(1) == self.read(2) else 0)

raw = contents(2019, 5, 1)[0]
ops = map_ints(raw.split(','))

@timeit
def day5():
    @timeit
    def part1():
        p1 = Machine(ops, [1])
        p1.run()
    @timeit
    def part2():
        p2 = Machine(ops, [5])
        p2.run()
    
    part1()
    part2()
day5()

0
0
0
0
0
0
0
0
0
7265618
halt:  3
'part1'  1.07 ms
7731427
halt:  314
'part2'  0.42 ms
'day5'  1.53 ms


In [62]:
"""
Day 4

1864
1258
'day4'  535.50 ms
"""

low = 137683
high = 596253

@timeit
def day4():
    part1 = 0
    part2 = 0
    
    for i in range(low, high + 1):
        s = f"{i}"
        double = False
        decrease = False
        for a, b in zip(s, s[1:]):
            if a == b: 
                double = True
            elif int(b) < int(a): 
                decrease = True
                break
        if double and not decrease:
            part1 += 1
            counter = Counter(s)
            if 2 in counter.values():
                part2 += 1
                
    print(part1)
    print(part2)
    
day4()

1864
1258
'day4'  541.21 ms


In [63]:
"""
Day 3

489
93654
'day3'  625.63 ms
"""

@timeit
def day3():
    raw = contents(2019, 3, 1)
    grid = defaultdict(dict)

    operations = {
        'R': xy_RIGHT,
        'L': xy_LEFT,
        'U': xy_UP,
        'D': xy_DOWN,
    }

    for wire, line in enumerate(raw):
        steps = 0
        current = origin
        for op in line.split(','):
            direction, distance = op[0], int(op[1:])
            for i in range(distance):
                steps += 1
                current = add(current, operations[direction])
                grid[current][wire] = grid[current][wire] if wire in grid[current] else steps
                
    print(min([cityblock_distance(pos) for pos, wires in grid.items() if len(wires.keys()) > 1]))
    print(min([sum(wires.values()) for pos, wires in grid.items() if len(wires.keys()) > 1]))
    
day3()

489
93654
'day3'  653.57 ms


In [64]:
"""
Day 2

3101844
'part1'  0.00 ms
84 78 8478
'part2'  121.99 ms
'day2'  129.00 ms
"""

@timeit
def day2():
    from operator import mul, add
    raw = contents(2019, 2, 1)[0]
    ops = map_ints(raw.split(','))
    
    def compute(ops):
        ip = 0
        while True:
            op = ops[ip]
            if op == 99:
                return ops[0]
            p1, p2, r = ops[ip+1], ops[ip+2], ops[ip+3]
            func = add if op == 1 else mul
            ops[r] = func(ops[p1], ops[p2])
            ip += 4
            
    @timeit
    def part1():
        inp = ops[:]
        inp[1] = 12
        inp[2] = 2
        print(compute(inp))
    
    @timeit
    def part2():
        for noun in range(1, 100):
            for verb in range(1, 100):
                inp = ops[:]
                inp[1] = noun
                inp[2] = verb
                try:
                    result = compute(inp)
                except IndexError:
                    result = -1
                if result == 19690720:
                    print(noun, verb, 100*noun+verb)
                    return
    
    part1()
    part2()
        
day2()

3101844
'part1'  0.39 ms
84 78 8478
'part2'  98.03 ms
'day2'  104.81 ms


In [65]:
"""
Day 1

Part 1: 3268951
'part1'  1.00 ms
Part 2: 4900568
'part2'  0.00 ms
'day1'  1.00 ms
"""

@timeit
def day1():
    
    masses = map_ints(contents(2019, 1, 1))
    
    @lru_cache
    def fuel_for_module(amount):
        return amount // 3 - 2
    
    @timeit
    def part1():
        fuel = sum(fuel_for_module(m) for m in masses)
        print(f'Part 1: {fuel}')
    
    @timeit
    def part2():
        @lru_cache
        def full_fuel_for_module(mass):
            total = 0
            while mass > 0:
                mass = fuel_for_module(mass)
                total += 0 if mass < 0 else mass
            return total
        fuel = sum(full_fuel_for_module(m) for m in masses)
        print(f'Part 2: {fuel}')
    
    part1()
    part2()
    
day1()

Part 1: 3268951
'part1'  0.16 ms
Part 2: 4900568
'part2'  0.42 ms
'day1'  6.87 ms
